## Auxillary code for generating cluster algebra data

This code is simply to generate the cluster algebra data we need for our machine learning. We use heavily the Sage cluster algebra package https://doc.sagemath.org/html/en/reference/algebras/sage/algebras/cluster_algebra.html which has an additional compendium https://arxiv.org/abs/1102.4844. This work is indepted to the papers [1,2] which served as an inspiration for this project.

<cite data-cite="bao">[1] Bao, Jiakang, et al. "Quiver mutations, Seiberg duality, and machine learning." Physical Review D 102.8 (2020): 086013.</cite> https://arxiv.org/abs/2006.10783

<cite data-cite="dechant">[2] Dechant, Pierre-Philippe, et al. "Cluster Algebras: Network Science and Machine Learning." arXiv preprint arXiv:2203.13847 (2022).

In [4]:
# We need this auxillary function to make lists with unique elements. Apparently we can't use sets because of hashability.

def check_not_in(list, element):
    '''This checks if element is not in list. 
       Returns False if element is in list, True otherwise'''
    for item in list:
        if element == item:
            return False
    return True

We first construct a function to reccursively search for new cluster seeds. The arguments are an initial seed and a depth $d$: we apply all mutation sequences of length $\leq d$ to the initial seed and collect all obtained seeds. 

In [2]:
def seeds(seed, depth):
    '''This gives all of the seeds in the mutation class of the initial seed, up to a given depth.
       Inputs: The initial seed and the depth to check to
       Outputs: The list of seeds'''
    n = seed.n() # The cluster size
    total_check = (n^depth-1) / (n-1) # This is the number of seeds we actually check, as a function of the depth
    seeds = []
    seeds_to_check = [seed]
    checked_seeds = []
    for i in range(1, total_check+1):
        current_seed = seeds_to_check[0] # Take the first seed to check
        if check_not_in(seeds, current_seed): # If not in seeds, we add it to the list
            seeds.append(current_seed)
        for j in range(0, n): # Now we look at the neighbours of current_seed
            if check_not_in(seeds, current_seed.mutate(j, inplace=False)) and check_not_in(seeds_to_check, current_seed.mutate(j, inplace=False)): # If the neighbour isn't in seeds or
                seeds_to_check.append(current_seed.mutate(j, inplace=False)) # seeds_to_check then we add to seeds_to_check
        seeds_to_check.pop(0) # Finished checking current_seed, so remove it from seeds_to_check
        if seeds_to_check == []:
            break
    return seeds

We also need a function to just calculate the $b$-matrices. This is faster since it discards all information about cluster variables.

In [8]:
def quivers(init_quiver, depth):
    '''This gives all of the quivers in the mutation class of the initial, up to a given depth.
       Inputs: The initial quiver and the depth to check to
       Outputs: The list of quivers'''
    n = init_quiver.n() # The number of vertices
    total_check = (n^depth-1) / (n-1) # This is the number of seeds we actually check, as a function of the depth
    quivers = []
    quivers_to_check = [init_quiver]
    checked_quivers = []
    for i in range(1, total_check+1):
        print(f'Checked {i} of {total_check}')
        current_quiver = quivers_to_check[0] # Take the first quiver to check
        if check_not_in(quivers, current_quiver): # If not in quivers, we add it to the list
            quivers.append(current_quiver)
        for j in range(0, n): # Now we look at the neighbours of current_quiver
            if check_not_in(quivers, current_quiver.mutate(j, inplace=False)) and check_not_in(quivers_to_check, current_quiver.mutate(j, inplace=False)):
                quivers_to_check.append(current_quiver.mutate(j, inplace=False))  
        quivers_to_check.pop(0) 
        if quivers_to_check == []:
            return quivers
    return quivers

def b_matrices(init_quiver, depth):
    quiver_set = quivers(init_quiver, depth)
    return [quiver_set[i].b_matrix() for i in range(0, len(quiver_set))]

For export we convert the $b$-matrices to vectors. We also need to keep track of which cluster algebra the matrix came from.

In [4]:
import csv

def export_b_matrices(seed, seed_name, depth):
    ''' This function converts the b matrices in the cluster algebra to a vector, and appends a label seed_name.
        It then exports to a csv file
    '''
    data = b_matrices(seed, depth)
    tagged_vector_data = [f'{seed_name}'] + [data[i].list() for i in range(0, len(data))]
    with open(f'cluster_data_{seed_name}_depth_{depth}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(tagged_vector_data)

In [1]:
# export_b_matrices(ClusterQuiver(['A', 6]), 'A6', 6)

In [2]:
# export_b_matrices(ClusterQuiver(['D', 6]), 'D6', 6)

In [3]:
# export_b_matrices(ClusterQuiver(['E', 6]), 'E6', 6)